1. Nombre total d'hectares en 2001 et en 2020 dans tous les tropiques (et pourcentage que ca représente)
2. Pareil mais par sl (roi)
3. Pareil par roi mais en 2050 et 2100


In [2]:
import pandas as pd 
import numpy as np 
import sys
sys.path.append('..')
from utils import d_sl, years, dense_vegetation, sl_tropics, version, compute_area_hectares, DATA_PATH
from produce_maps.tcrit import TCRIT, Tcrit_map, data_both_biomes
import rasterio as rio 

file_species = DATA_PATH + f'/outputs/species_above_threshold_1981_2010{version}.tif'
rio_species = rio.open(file_species, 'r')
data_allspecies = rio_species.read(1)
data_allspecies[dense_vegetation == 0] = 0
data_allspecies[~data_both_biomes] = 0
d_ext = {"South America": "south_america",  "Africa": "africa", "SE Asia": "south_east_asia"}
d_ext = {value: key for key, value in d_ext.items()}

In [2]:
# 2020
total = 0
total_above = 0
d_above_2020 = {}
for key, sl in list(d_sl.items()):
    ext = key
    color = 'tab:red' #d_color[key]
    count_species = data_allspecies[sl]
    outfile = f"/data/gaston/ecostress/ecostress/outputs/delta_Tmin_Tmodis{ext}_per_year_2001_2020_reversed{version}.npy"
    map = np.load(outfile)
    map = map.astype(float)[..., -1] # 2020
    map[~data_both_biomes[sl]] = np.nan
    above = compute_area_hectares(map > 0, d_sl[key])
    tot = compute_area_hectares(~np.isnan(map), d_sl[key])
    total += tot 
    total_above += above
    print(key, above/1000000, tot/1000000, (above/tot)*100)
    d_above_2020[key] = above/1000000
    
print("total", total_above/1000000, total/1000000, (total_above/total)*100)


# south_east_asia 27.11138761965745 435.3275798456408 6.227813002169689
# south_america 138.11514600718036 773.1341649591459 17.864318027451116
# africa 8.895827595225741 182.49401550897488 4.874585925689303
# total 174.12236122206357 1390.9557603137614 12.518181108994176

south_east_asia 27.158382795639923 435.69943244637034 6.233283950624107
south_america 138.10930424777985 773.1337376042538 17.86357230713352
africa 8.895827595225741 182.49401550897488 4.874585925689303
total 174.16351463864552 1391.3271855595992 12.51779713975732


In [3]:
# 2001
total = 0
total_above = 0
for key, sl in list(d_sl.items()):
    ext = key
    color = 'tab:red' #d_color[key]
    count_species = data_allspecies[sl]
    outfile = DATA_PATH + f"/outputs/delta_Tmin_Tmodis{ext}_per_year_2001_2020_reversed{version}.npy"
    map = np.load(outfile)
    map = map.astype(float)[..., 0] # 2001
    map[~data_both_biomes[sl]] = np.nan
    above = compute_area_hectares(map > 0, d_sl[key])
    tot = compute_area_hectares(~np.isnan(map), d_sl[key])
    total += tot 
    total_above += above
    print(key, above/1000000, tot/1000000, (above/tot)*100)
print("total", total_above/1000000, total/1000000, (total_above/total)*100)

# south_east_asia 22.252639609697297 435.3275798456408 5.111699933550655
# south_america 105.77935991453428 773.1341649591459 13.681889212608253
# africa 10.222198980402956 182.49401550897488 5.601388600000551
# total 138.25419850463453 1390.9557603137614 9.939510834869987

south_east_asia 22.31092339118828 435.69943244637034 5.120714357123819
south_america 105.76848512800042 773.1337376042538 13.680490189931465
africa 10.222198980402956 182.49401550897488 5.601388600000551
total 138.30160749959165 1391.3271855595992 9.940264873352993


In [4]:
# fixed distribution 2050
ssp = '2041_2070_ssp370'
total = 0
total_above = 0
d_above_2050 = {}
for key, sl in list(d_sl.items()):
    ext = key
    sl = d_sl[key]
    
    y = '2050'
    filename_delta = DATA_PATH + f"/2050-2100_temperatures/delta_tsurf_2020_{y}.tif"
    delta_tsurf = rio.open(filename_delta, 'r').read(1)[sl]
    delta_tsurf[delta_tsurf==-20] = np.nan
    delta_tsurf[~data_both_biomes[sl]] = np.nan


    outfile = DATA_PATH + f"/outputs/delta_Tmin_Tmodis{ext}_per_year_2001_2020_reversed{version}.npy"
    # nan_map = np.isnan(np.load(outfile).astype(float)[..., 0])
    map = np.load(outfile)
    map = map.astype(float)[..., -1] # 2020
    map[~data_both_biomes[sl]] = np.nan
    map = map + delta_tsurf
    values = - map
    # values = np.load(f"/media/data/gaston/ecostress/outputs/LinearRegression_delta_Tmin_Tmodis{ext}_2050_tminv6_{ssp}.npy")
    # values = values - delta_tsurf
    # values[nan_map] = np.nan
    # values = values.flatten()
    # values = values[~np.isnan(values)]
    above = compute_area_hectares(values < 0, d_sl[key])
    tot = compute_area_hectares(~np.isnan(values), d_sl[key])
    total += tot 
    total_above += above
    print(key, above/1000000, tot/1000000, (above/tot)*100)
    d_above_2050[key] = above/1000000
print("total", total_above/1000000, total/1000000, (total_above/total)*100)

# south_east_asia 36.02915349791185 435.3273248737554 8.276336319655623
# south_america 168.37814709445456 773.1328331874156 21.778682765324742
# africa 11.299990420133764 182.49401550897488 6.191978618377237
# total 215.70729101250018 1390.954173570146 15.50786468103739

south_east_asia 36.0995490913424 435.6991774744847 8.28542970877113
south_america 168.37005517352952 773.1324058325209 21.777648162636005
africa 11.299990420133764 182.49401550897488 6.191978618377237
total 215.76959468500573 1391.3255988159804 15.50820274338558


In [5]:
# fixed distribution 2050
ssp = '2041_2070_ssp370'
total = 0
total_above = 0
d_above_2050_accl = {}
print("Taking acclimation into account")
for key, sl in list(d_sl.items()):
    ext = key
    sl = d_sl[key]
    
    y = '2050'
    filename_delta = DATA_PATH + f"/2050-2100_temperatures/delta_tsurf_2020_{y}.tif"
    delta_tsurf = rio.open(filename_delta, 'r').read(1)[sl]
    delta_tsurf[delta_tsurf==-20] = np.nan
    delta_tsurf[~data_both_biomes[sl]] = np.nan


    outfile = DATA_PATH + f"/outputs/delta_Tmin_Tmodis{ext}_per_year_2001_2020_reversed{version}.npy"
    # nan_map = np.isnan(np.load(outfile).astype(float)[..., 0])
    map = np.load(outfile)
    map = map.astype(float)[..., -1] # 2020
    map[~data_both_biomes[sl]] = np.nan
    map = map + delta_tsurf
    values = - map
    values += (delta_tsurf * 0.38)
    above = compute_area_hectares(values < 0, d_sl[key])
    tot = compute_area_hectares(~np.isnan(values), d_sl[key])
    total += tot 
    total_above += above
    print(key, above/1000000, tot/1000000, (above/tot)*100)
    d_above_2050_accl[key] = above/1000000
print("total", total_above/1000000, total/1000000, (total_above/total)*100)

# Taking acclimation into account
# south_east_asia 32.350550249746746 435.3273248737554 7.4313162536094834
# south_america 156.09765632641614 773.1328331874156 20.190276447433764
# africa 10.338918335156082 182.49401550897488 5.66534650811474
# total 198.78712491131895 1390.954173570146 14.291421578691867


Taking acclimation into account
south_east_asia 32.41032414773805 435.6991774744847 7.438692984366732
south_america 156.0900352001232 773.1324058325209 20.189301861179008
africa 10.338918335156082 182.49401550897488 5.66534650811474
total 198.83927768301731 1391.3255988159804 14.291354795184516


In [6]:
print("total", total_above/1000000, total/1000000, (total_above/total)*100)
# total 198.78712491131895 1390.954173570146 14.291421578691867

total 198.83927768301731 1391.3255988159804 14.291354795184516


In [7]:
# fixed distribution 2100
ssp = '2071_2100_ssp370'
total = 0
total_above = 0
d_above_2100 = {}
for key, sl in list(d_sl.items()):
    ext = key
    sl = d_sl[key]
    
    y = '2100'
    filename_delta = DATA_PATH + f"/2050-2100_temperatures/delta_tsurf_2020_{y}.tif"
    delta_tsurf = rio.open(filename_delta, 'r').read(1)[sl]
    delta_tsurf[delta_tsurf==-20] = np.nan
    delta_tsurf[~data_both_biomes[sl]] = np.nan


    outfile = outfile = DATA_PATH + f"/outputs/delta_Tmin_Tmodis{ext}_per_year_2001_2020_reversed{version}.npy"
    map = np.load(outfile)
    map = map.astype(float)[..., -1] # 2020
    map[~data_both_biomes[sl]] = np.nan
    map = map + delta_tsurf
    values = - map
    above = compute_area_hectares(values < 0, d_sl[key])
    tot = compute_area_hectares(~np.isnan(values), d_sl[key])
    total += tot 
    total_above += above
    print(key, above/1000000, tot/1000000, (above/tot)*100)
    d_above_2100[key] = above/1000000
print("total", total_above/1000000, total/1000000, (total_above/total)*100)

# south_east_asia 74.52581014518758 435.3273248737554 17.11948823033335
# south_america 261.04738393011564 773.1328331874156 33.76488136636088
# africa 20.397343383213258 182.49401550897488 11.176993024305576
# total 355.9705374585165 1390.954173570146 25.591823528222434

south_east_asia 74.72581862566749 435.6991774744847 17.150782578661982
south_america 261.03649542877184 773.1324058325209 33.76349166837518
africa 20.397343383213258 182.49401550897488 11.176993024305576
total 356.1596574376526 1391.3255988159804 25.598584381739602


In [8]:
# fixed distribution 2100
ssp = '2071_2100_ssp370'
total = 0
total_above = 0
d_above_2100_accl = {}
print("Taking acclimation into account")
for key, sl in list(d_sl.items()):
    ext = key
    sl = d_sl[key]
    
    y = '2100'
    filename_delta = DATA_PATH + f"/2050-2100_temperatures/delta_tsurf_2020_{y}.tif"
    delta_tsurf = rio.open(filename_delta, 'r').read(1)[sl]
    delta_tsurf[delta_tsurf==-20] = np.nan
    delta_tsurf[~data_both_biomes[sl]] = np.nan


    outfile = outfile = DATA_PATH + f"/outputs/delta_Tmin_Tmodis{ext}_per_year_2001_2020_reversed{version}.npy"
    map = np.load(outfile)
    map = map.astype(float)[..., -1] # 2020
    map[~data_both_biomes[sl]] = np.nan
    map = map + delta_tsurf
    values = - map
    values += (delta_tsurf * 0.38)
    above = compute_area_hectares(values < 0, d_sl[key])
    tot = compute_area_hectares(~np.isnan(values), d_sl[key])
    total += tot 
    total_above += above
    print(key, above/1000000, tot/1000000, (above/tot)*100)
    d_above_2100_accl[key] = above/1000000
print("total", total_above/1000000, total/1000000, (total_above/total)*100)

# Taking acclimation into account
# south_east_asia 50.79094202461453 435.3273248737554 11.667299322261442
# south_america 197.2940723631662 773.1328331874156 25.51878071841505
# africa 14.945170925439978 182.49401550897488 8.18940329838103
# total 263.0301853132207 1390.954173570146 18.910053998263955

Taking acclimation into account
south_east_asia 50.89472343304961 435.6991774744847 11.681161237911699
south_america 197.28466721456593 773.1324058325209 25.517578325038226
africa 14.945170925439978 182.49401550897488 8.18940329838103
total 263.12456157305553 1391.3255988159804 18.911789001580566


In [9]:
for key in d_sl.keys():
    print(key, d_above_2050[key]- d_above_2020[key], d_above_2100[key] - d_above_2020[key])
    print(key, d_above_2050_accl[key]- d_above_2020[key], d_above_2100_accl[key] - d_above_2020[key])
    
# south_east_asia 8.9177658782544 47.41442252553013
# south_america 30.263001087274205 122.93223792293529
# africa 2.4041628249080222 11.501515787987517

south_east_asia 8.94116629570248 47.56743583002756
south_east_asia 5.251941352098129 23.73634063740969
south_america 30.260750925749676 122.927191180992
south_america 17.98073095234335 59.17536296678608
africa 2.4041628249080222 11.501515787987517
africa 1.4430907399303408 6.049343330214237


In [10]:
# Tcrit min et max
Tmin = rio.open(DATA_PATH + f"outputs/Tcrit_map_min_1981_2010{version}.tif", 'r').read(1)
Tmin[Tmin==1000] = np.nan
Tmin[Tmin==0] = np.nan
Tmax = rio.open(DATA_PATH + f"outputs/Tcrit_map_max_1981_2010{version}.tif", 'r').read(1)
Tmax[Tmax==0] = np.nan
Tmin[~data_both_biomes] = np.nan
Tmax[~data_both_biomes] = np.nan
for key, sl in d_sl.items():
    print(np.nanmin(Tmin[sl]), np.nanmax(Tmax[sl]))
    # print(np.nanmin(Tmax[sl]), )
    
# 40.01 49.7
# 31.05 54.9583636836448
# 41.58 53.618001241091626

40.01 44.245
31.05 nan
41.58 nan


/tmp/ipykernel_649357/472777037.py:10: RuntimeWarning: All-NaN slice encountered
  print(np.nanmin(Tmin[sl]), np.nanmax(Tmax[sl]))


In [11]:
a= []
for ext, sl in d_sl.items():
    lr_outfile = f"/data/gaston/ecostress/ecostress/outputs/LinearRegression_delta_Tmin_Tmodis{ext}_per_year_2001_2020{version}.npy"
    lr_map = - np.load(lr_outfile)
    lr_map[~data_both_biomes[sl]] = np.nan
    a.append(lr_map[~np.isnan(lr_map)].flatten())
    print(ext, np.nanmean(lr_map))
print(np.nanmean(np.concatenate(a)))

# south_east_asia -0.01927870443704044
# south_america -0.0475145019220043
# africa -0.03325496459614932
# -0.03669079717015983

south_east_asia -0.01928791648707466
south_america -0.047514517249509974
africa -0.03325496459614932
-0.03668881877277814


In [12]:
a= []
for ext, sl in d_sl.items():
    outfile = DATA_PATH + f"/outputs/delta_Tmin_Tmodis{ext}_per_year_2001_2020_reversed{version}.npy"
    map = np.load(outfile)
    map[~data_both_biomes[sl]] = np.nan
    a.append(map[~np.isnan(map)].flatten())
    map = (map[...,0] + Tmin[sl])
    print(ext, np.nanpercentile(map,99)) 
print(np.nanquantile(np.concatenate(a), 0.6))
print(np.nanquantile(np.concatenate(a), 0.75))

# 46.42999244689941
# 48.79000831604004
# 49.089996334910396
# -7.569994926452637
# -5.179996490478516

south_east_asia 46.45001197814941
south_america 48.79000831604004
africa 49.089996334910396
-7.560003757476807
-5.170007228851318


In [13]:
total = 0
total_above = 0
for ext, sl in d_sl.items():
    key = ext
    lr_outfile = f"/data/gaston/ecostress/ecostress/outputs/LinearRegression_delta_Tmin_Tmodis{ext}_per_year_2001_2020{version}.npy"
    lr_map = - np.load(lr_outfile)
    lr_map[~data_both_biomes[sl]] = np.nan
    above = compute_area_hectares(lr_map >= 0, d_sl[key])
    tot = compute_area_hectares(~np.isnan(lr_map), d_sl[key])
    total += tot 
    total_above += above
    print(key, above/1000000, tot/1000000, (above/tot)*100)
    
# south_east_asia 163.00445912990077 435.3275798456408 37.44409191526509
# south_america 187.6968413446885 773.1341649591459 24.27739580679465
# africa 57.387100549917044 182.49401550897488 31.446017772070338

south_east_asia 163.116626401837 435.69943244637034 37.4378790181038
south_america 187.6968413446885 773.1337376042538 24.27740922628905
africa 57.387100549917044 182.49401550897488 31.446017772070338


In [14]:
for ext, sl in d_sl.items():
    print(ext, data_both_biomes[sl].shape)
    
# south_east_asia (5018, 9999)
# south_america (7104, 9122)
# africa (4497, 7693)

south_east_asia (5018, 9999)
south_america (7104, 9122)
africa (4497, 7693)


## Tair evolution

In [3]:
total = 0
total_above = 0
total_neg = np.array([])
total_pos = np.array([])
for ext, sl in d_sl.items():
    key = ext
    lr_outfile = DATA_PATH + f"/outputs/LinearRegression_delta_Tmin_Tmodis{ext}_per_year_2001_2020{version}.npy"
    lr_map = - np.load(lr_outfile)
    lr_map[~data_both_biomes[sl]] = np.nan
    lr_tair_outfile = DATA_PATH + f"/outputs/LinearRegression_Tair{ext}_per_year_2001_2020_{version}.npy"
    lr_tair_map = np.load(lr_tair_outfile)
    total_pos = np.concatenate([total_pos, lr_tair_map[lr_map > 0]])
    total_neg = np.concatenate([total_neg, lr_tair_map[lr_map < 0]])
    print(key, "Tair with TSM up:", np.nanmean(lr_tair_map[lr_map >= 0]), "Tair with TSM down", np.nanmean(lr_tair_map[lr_map < 0]), "TSM with TSM up:", np.nanmean(lr_map[lr_map >= 0]), "TSM with TSM down", np.nanmean(lr_map[lr_map < 0]))
    
print(np.nanmean(total_pos), np.nanmean(total_neg))

# south_east_asia Tair with TSM up: 0.012552211603656195 Tair with TSM down 0.022591978758514267 TSM with TSM up: 0.05837295455580159 TSM with TSM down -0.06636507397318372
# south_america Tair with TSM up: 0.03839570853311328 Tair with TSM down 0.0654038379299129 TSM with TSM up: 0.04024279103568656 TSM with TSM down -0.07572393262402081
# africa Tair with TSM up: 0.04306477099559266 Tair with TSM down 0.051144659033577504 TSM with TSM up: 0.05378082312177608 TSM with TSM down -0.0729809795947593
# 0.028515201184681857 0.051584120402371224

south_east_asia Tair with TSM up: 0.012557318211087642 Tair with TSM down 0.0226008458786825 TSM with TSM up: 0.05838004112725992 TSM with TSM down -0.06637177072530198
south_america Tair with TSM up: 0.03839570853311328 Tair with TSM down 0.06540381765694055 TSM with TSM up: 0.04024279103561395 TSM with TSM down -0.07572397308820976
africa Tair with TSM up: 0.04306477099559266 Tair with TSM down 0.051144659033577504 TSM with TSM up: 0.05378082312177608 TSM with TSM down -0.0729809795947593
0.028512579676834904 0.051578557115092315
